### Query MySQL for data

In [1]:
import pymysql

# Connect to the database
connection = pymysql.connect(host='127.0.0.1',
                             user='root',
                             port=3306,
                             db='db',
                             password='password')

cursor = connection.cursor()

print('user table size: ' + str(cursor.execute("SELECT * FROM db.users")))
print('statuses table size: ' + str(cursor.execute("SELECT * FROM db.statuses")))
print('edgelist table size: ' + str(cursor.execute("SELECT * FROM db.edgelist")))

user table size: 1090
statuses table size: 208585
edgelist table size: 501819


## Find the Most Liked Tweets

In [2]:
import pandas as pd
import network_vis_helper as visnet
import time
from datetime import timedelta
pd.set_option('display.max_colwidth', None)

start_time = time.time()
statuses = pd.read_sql("SELECT * FROM statuses", connection)
most_liked_tweets = (statuses[['author_screen_name', 'text', 'like_count']].
                     sort_values('like_count', ascending = False).
                     head(10))
print("--- %s time elapsed ---" % str(timedelta(seconds=time.time() - start_time)))
most_liked_tweets

--- 0:00:05.144868 time elapsed ---


,author_screen_name,text,like_count
125780,UNICEF,"💜💙\n\n@BTS_twt s messages of solidarity have given hope to children and young people around the world during COVID-19.\n\nToday, we are proud to announce that we are renewing our joint commitment to creating a kinder, safer and more connected world. Are you with us?\n\n#BTSLoveMyself",172265
125781,UNICEF,"The first step to creating a kinder world? Learning to love yourself.\n \nAs we face the second year of the COVID-19 pandemic, @bts_twt reflect on the importance of connection and self-acceptance.\n \n#BTSLoveMyself",118748
129936,UNICEF,"⚡️ A look back on the #BTSLoveMyself campaign, as @BTS_twt renew their commitment to helping UNICEF #ENDviolence and promote self-esteem and well-being.\n\nhttps://t.co/Z5BIGJ12j1",38483
64592,Surgeon_General,"I ve been asked by the Biden team to step down as Surgeon General. Its been the honor of my life to serve this Nation, and I will do all I can to ensure everyone has an equal opportunity to achieve and maintain health. See my full statement at: https://t.co/pCkbpealt8",16558
26582,forrestmaready,It s just a mask?\n\nIt s just two weeks.\nIt s just non-essential businesses.\nIt s just to keep from overwhelming the hospitals.\nIt s just until cases go down more.\nIt s just to keep others from being scared.\nIt s just for a few more weeks.,15369
184514,PeterHotez,"Looking for things to criticize or question in the @JoeBiden speech tonight. For me, nada. A home run. Thank you. \n\nTurns out we elected a really good POTUS.",13758
104850,aprillynnclark,https://t.co/bY60vqhxVI,11918
64865,CDCDirector,"It is a privilege to join @CDCgov. For 75 years, CDC has carried out a mission to protect America’s safety, health, &amp; security at home &amp; abroad. I recognize the seriousness of the moment - the toll that #COVID19 has had on America is truly heartbreaking. https://t.co/NLVqme863t",11386
65336,CDCDirector,"Although I never anticipated leading the response to #COVID19, when the nation is coding, if you are called to serve, you serve. You run to take care of people, to give them hope and help them have a fighting chance to come back stronger.",10042
73406,RichardReichle,Why aren t the unmasked dead by now?,9238


Now compare to the speed of asking the same question using SQL.

In [3]:
start_time = time.time()
most_liked_tweets = pd.read_sql("SELECT author_screen_name, text, like_count FROM statuses ORDER BY like_count DESC LIMIT 10", connection) 
print("--- %s time elapsed ---" % str(timedelta(seconds=time.time() - start_time)))
most_liked_tweets

--- 0:00:00.162069 time elapsed ---


,author_screen_name,text,like_count
0,UNICEF,"💜💙\n\n@BTS_twt s messages of solidarity have given hope to children and young people around the world during COVID-19.\n\nToday, we are proud to announce that we are renewing our joint commitment to creating a kinder, safer and more connected world. Are you with us?\n\n#BTSLoveMyself",172265
1,UNICEF,"The first step to creating a kinder world? Learning to love yourself.\n \nAs we face the second year of the COVID-19 pandemic, @bts_twt reflect on the importance of connection and self-acceptance.\n \n#BTSLoveMyself",118748
2,UNICEF,"⚡️ A look back on the #BTSLoveMyself campaign, as @BTS_twt renew their commitment to helping UNICEF #ENDviolence and promote self-esteem and well-being.\n\nhttps://t.co/Z5BIGJ12j1",38483
3,Surgeon_General,"I ve been asked by the Biden team to step down as Surgeon General. Its been the honor of my life to serve this Nation, and I will do all I can to ensure everyone has an equal opportunity to achieve and maintain health. See my full statement at: https://t.co/pCkbpealt8",16558
4,forrestmaready,It s just a mask?\n\nIt s just two weeks.\nIt s just non-essential businesses.\nIt s just to keep from overwhelming the hospitals.\nIt s just until cases go down more.\nIt s just to keep others from being scared.\nIt s just for a few more weeks.,15369
5,PeterHotez,"Looking for things to criticize or question in the @JoeBiden speech tonight. For me, nada. A home run. Thank you. \n\nTurns out we elected a really good POTUS.",13758
6,aprillynnclark,https://t.co/bY60vqhxVI,11918
7,CDCDirector,"It is a privilege to join @CDCgov. For 75 years, CDC has carried out a mission to protect America’s safety, health, &amp; security at home &amp; abroad. I recognize the seriousness of the moment - the toll that #COVID19 has had on America is truly heartbreaking. https://t.co/NLVqme863t",11386
8,CDCDirector,"Although I never anticipated leading the response to #COVID19, when the nation is coding, if you are called to serve, you serve. You run to take care of people, to give them hope and help them have a fighting chance to come back stronger.",10042
9,RichardReichle,Why aren t the unmasked dead by now?,9238


### Visualize networks

In [4]:
el = pd.read_sql("SELECT * FROM edgelist WHERE edge_type = 'hashtag'", connection)
weighted_el = visnet.get_weighted_el(el)
visnet.render_graph(weighted_el, edge_type = 'hashtag', edge_weight_threshold = 20)

In [5]:
top_hashtags = (weighted_el[weighted_el['edge_type'] == 'hashtag']
                .groupby('target').sum().reset_index()
                .sort_values('weight', ascending = False))
top_hashtags.head(20)

,target,weight
2454,covid19,4487
11961,vaccine,916
12030,vaccines,818
2452,covid,763
12065,vaccineswork,746
5093,health,577
2371,coronavirus,468
2550,covidvaccine,419
237,adoptdontshop,392
5101,healthcare,363
